In [53]:
#importing required modules
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression

In [54]:
#importing Library
import os

#printing the current directory
datapath = os.getcwd()

In [55]:
##attaching the file to the datapath
file_path = datapath + "\\DecayTimecourse.txt"
file_path

'C:\\Users\\saini\\OneDrive\\Desktop\\IUPUI\\Computational\\Assignment_2\\DecayTimecourse.txt'

In [56]:
#Reading the DecayTimecourse text file
raw_data = pd.read_csv(file_path, sep = "\t", skiprows = 1)

In [57]:
#renaming the first column as "gene_name"
raw_data.rename(columns={ raw_data.columns[0]: "gene_name" }, inplace = True)

In [58]:
#Total number of unique genes in the dataframe
raw_data.gene_name.nunique()

6183

In [90]:
#defining a function to fit the data and calculate the half life
def get_half_life(gene_list,df1):
    
    #creating an empty list
    gene_slope_list = []

    for gene in gene_list: #writing a for loop to calculate half life for each gene and concate the resulting dataframe
        df = df1[df1.gene_name == gene] #subsetting for each gene 
        df = df.reset_index(drop = True) 
        time_steps = list(df.columns) #making a list of all columns
        time_steps = time_steps[1:] #list of given time intervals
        gene_exp = df.values.tolist()[0] #converting the gene expression values to a list
        gene_exp = gene_exp[1:] #excluding the first column
        gene_exp_df = pd.DataFrame({'X': time_steps,'y': gene_exp}) #creating a data frame with X being the time intervals and y = gene expression levels
        gene_exp_df = gene_exp_df.dropna(axis = 0) #dropping the time intervals with null values
        gene_exp_df['y'] = gene_exp_df['y'].replace(0, 0.000001) #If the data has 0, it is replaced by 0.000001 as ln(0) is infinity
        if len(gene_exp_df) > 0:
        
            min_val = min(gene_exp_df['y'])
            if min_val <= 0:
                log_contant = 1 - min_val
                gene_exp_df['y'] = gene_exp_df['y'] + log_contant

            gene_exp_df['y'] = np.log(gene_exp_df.y)
            X = gene_exp_df['X'] #defining X
            y = gene_exp_df['y'] #defining y
            X = gene_exp_df.iloc[:, 0].values.reshape(-1, 1)  #values are converted into a numpy array
            y = gene_exp_df.iloc[:, 1].values.reshape(-1, 1) #-1 means that calculate the dimension of rows, but have 1 column

            reg = LinearRegression().fit(X,y) #fitting the values to linear regression model
            reg.intercept_ = reg.intercept_ + 0.00000000001 #to avoid diving by zero

            half_life = np.log(2)/(reg.intercept_) #calculating the t1/2 using formula ln(2)/slope(k)

            save_gene = pd.DataFrame(columns = ['gene','half_life']) #creating a data frame with 2 columns
            save_gene["half_life"] = half_life #one of the column has half_life value
            save_gene["gene"] = str(gene) #another column has the gene name
            gene_slope_list.append(save_gene) 
    t_half_gene = pd.concat(gene_slope_list)
    
    return t_half_gene

In [91]:
#defining the first data frame
df1 = raw_data.iloc[:,0:10]

In [92]:
#listing of all the genes from data frame 1
gene_list1 = df1.gene_name.unique()

In [93]:
#calling the defined function to calculate half life
t_half1_gene = get_half_life(gene_list1, df1)

In [94]:
#defining the second data frame for df1
df2 = pd.concat([raw_data['gene_name'],raw_data.iloc[:,10:19]], axis = 1)

In [95]:
#listing of all the genes from data frame 2
gene_list2 = df2.gene_name.unique()

In [96]:
#calling the defined function to calculate half life
t_half2_gene = get_half_life(gene_list2, df2)

In [97]:
#defining the third data frame for df2
df3 = pd.concat([raw_data['gene_name'],raw_data.iloc[:,19:28]], axis = 1)

In [98]:
#listing of all the genes from data frame 3
gene_list3 = df3.gene_name.unique()

In [99]:
#calling the defined function to calculate half life for df3
t_half3_gene = get_half_life(gene_list3, df3)

In [100]:
#merging the data frames of first and second time course half life
mer_col = pd.merge(t_half1_gene, t_half2_gene, on="gene", how="outer")

In [101]:
#merging the above dataframe with the third time course half life 
final = pd.merge(mer_col, t_half3_gene, on="gene", how="outer")

In [102]:
final.head(10)

,gene,half_life_x,half_life_y,half_life
0,YAL026C,-2.597672e+01,5.326105,NaN
1,YIL125W,1.158774e+01,-6.507640,-4.648626
2,YCL009C,-2.259472e+00,27.746494,4.825472
3,YMR108W,-7.906045e+00,-3.739159,-20.021897
4,YGL032C,8.890191e-01,NaN,6.275206
5,YMR261C,-8.089337e+00,109.002459,22.540463
6,YBR126C,2.244410e+00,-39.528962,3.078507
7,YDR074W,2.508034e+00,-1900.691869,10.640385
8,YML100W,6.931472e+10,-4.545919,-7.999784
9,YPL074W,-2.649500e+01,NaN,-3.392640


In [103]:
#calculating the average of obtained half lives
final['avg_half_life'] = final.mean(axis=1)

In [104]:
final

,gene,half_life_x,half_life_y,half_life,avg_half_life
0,YAL026C,-25.976725,5.326105,NaN,-10.325310
1,YIL125W,11.587744,-6.507640,-4.648626,0.143826
2,YCL009C,-2.259472,27.746494,4.825472,10.104165
3,YMR108W,-7.906045,-3.739159,-20.021897,-10.555700
4,YGL032C,0.889019,NaN,6.275206,3.582112
...,...,...,...,...,...
6169,YMR326C,NaN,NaN,1.060324,1.060324
6170,YBR300C,NaN,NaN,1.125946,1.125946
6171,YHR211W,NaN,NaN,3.297014,3.297014
6172,YCR102C,NaN,NaN,1.100662,1.100662


In [105]:
#sorting the data frame based average half life in ascending order
sorted_df = final.sort_values(by=['avg_half_life'], ascending=True)
sorted_df

,gene,half_life_x,half_life_y,half_life,avg_half_life
3113,YOR288C,3.360757e+00,-1.186550e+01,-1.188089e+05,-3.960579e+04
2708,YLR270W,-3.800641e+04,-4.218219e+02,7.412784e+00,-1.280694e+04
5110,YKR015C,-2.970283e+01,-1.312971e+04,-1.529512e+01,-4.391569e+03
890,YML098W,-2.101161e+00,1.020393e+01,-1.079010e+04,-3.593997e+03
4765,YMR136W,4.997246e+00,-8.456071e+03,-3.195438e+00,-2.818090e+03
...,...,...,...,...,...
6035,YGR021W,NaN,NaN,6.931472e+10,6.931472e+10
1607,YFL014W,6.931472e+10,6.931472e+10,6.931472e+10,6.931472e+10
1935,YLL026W,6.931472e+10,6.931472e+10,6.931472e+10,6.931472e+10
4769,YMR250W,6.931472e+10,6.931472e+10,6.931472e+10,6.931472e+10


In [111]:
ten_percent_count = int(len(sorted_df)*.1)

617

In [112]:
#the genes with very low half lives (bottom 10%)
bottom_10_percent = sorted_df.head(ten_percent_count)

In [113]:
bottom_10_percent

,gene,half_life_x,half_life_y,half_life,avg_half_life
3113,YOR288C,3.360757,-11.865498,-118808.865915,-39605.790219
2708,YLR270W,-38006.411577,-421.821909,7.412784,-12806.940234
5110,YKR015C,-29.702833,-13129.708968,-15.295120,-4391.568973
890,YML098W,-2.101161,10.203930,-10790.095265,-3593.997499
4765,YMR136W,4.997246,-8456.070557,-3.195438,-2818.089583
...,...,...,...,...,...
876,YOL051W,-9.628833,-5.856801,-21.958855,-12.481497
3938,YOR226C,-32.604769,-0.709339,-4.113757,-12.475955
5846,YAR018C,NaN,-0.880166,-24.051361,-12.465764
2755,YPR077C,-5.571897,NaN,-19.328848,-12.450372


In [114]:
#the genes with very high half lives (top 10%)
top_10_percent = sorted_df.tail(ten_percent_count)

In [115]:
top_10_percent

,gene,half_life_x,half_life_y,half_life,avg_half_life
4650,YHR202W,-4.497438e+00,2.261384e+01,1.436512e+02,5.392253e+01
3175,YGL210W,1.624314e+02,-1.617813e+00,1.439187e+00,5.408426e+01
4058,YOL095C,5.302944e+00,2.281198e+02,-7.037258e+01,5.435005e+01
4374,YPR027C,1.008122e+00,1.640641e+02,-1.838200e+00,5.441133e+01
5516,YOR363C,NaN,1.037842e+02,6.385379e+00,5.508478e+01
...,...,...,...,...,...
6035,YGR021W,NaN,NaN,6.931472e+10,6.931472e+10
1607,YFL014W,6.931472e+10,6.931472e+10,6.931472e+10,6.931472e+10
1935,YLL026W,6.931472e+10,6.931472e+10,6.931472e+10,6.931472e+10
4769,YMR250W,6.931472e+10,6.931472e+10,6.931472e+10,6.931472e+10
